# **SQL**

<div style="border:solid Blue 2px; padding: 20px">

**Цель работы:** проанализировать информацию о книгах, издательстве, авторах и обзорах

**План работы:**
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Посчитать для каждой книги количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг(учитывать только книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

<div style="border:solid Orange 2px; padding: 20px">

**Описание данных:**

1. **Таблица books** содержит данные о книгах:
- **book_id** — идентификатор книги;
- **author_id** — идентификатор автора;
- **title** — название книги;
- **num_pages** — количество страниц;
- **publication_date** — дата публикации книги;
- **publisher_id** — идентификатор издателя.

2. **Таблица authors** содержит данные об авторах:
- **author_id** — идентификатор автора;
- **author** — имя автора.

3. **Таблица publishers** содержит данные об издательствах:
- **publisher_id** — идентификатор издательства;
- **publisher** — название издательства;

4. **Таблица ratings** содержит данные о пользовательских оценках книг:
- **rating_id** — идентификатор оценки;
- **book_id** — идентификатор книги;
- **username** — имя пользователя, оставившего оценку;
- **rating** — оценка книги.

5. **Таблица reviews** содержит данные о пользовательских обзорах на книги:
- **review_id** — идентификатор обзора;
- **book_id** — идентификатор книги;
- **username** — имя пользователя, написавшего обзор;
- **text** — текст обзора.

## **Загрузим библиотеки и установим параметры**

In [1]:
#Загрузим библиотеки
import pandas as pd
import sqlalchemy as sa
!pip install psycopg2-binary

In [2]:
#Устанавливаем параметры
db_config = {
            'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db' # название базы данных
            }
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

#Сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

<div style="border:solid Green 2px; padding: 20px">

**Вывод по блоку:**
- Загрузили библиотеки;
- Установили параметры.

## **Знакомимся с данными**

### Таблица books 

In [3]:
#Загрузим таблицу books
query = '''
            SELECT * 
            FROM books 
        '''
books = get_sql_data(query)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [4]:
#Посмотрим на данные
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- В таблице **books** 1000 строк;

### Таблица authors

In [5]:
#Загрузим таблицу authors
query = '''
            SELECT * 
            FROM authors 
        '''
authors = get_sql_data(query)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [6]:
#Посмотрим на данные
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- В таблице **authors** 636 строки;

### Таблица publishers

In [7]:
#Загрузим таблицу publishers
query = '''
            SELECT * 
            FROM publishers 
        '''
publishers = get_sql_data(query)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [8]:
#Посмотрим на данные
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- В таблице **publishers** 340 строк;

### Таблица ratings

In [9]:
#Загрузим таблицу ratings
query = '''
            SELECT * 
            FROM ratings 
        '''
ratings = get_sql_data(query)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [10]:
#Посмотрим на данные
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- В таблице **ratings** 6456 строк;

### Таблица reviews

In [11]:
#Загрузим таблицу reviews
query = '''
            SELECT * 
            FROM reviews 
        '''
reviews = get_sql_data(query)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


In [12]:
#Посмотрим на данные
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- В таблице **reviews** 2793 строк;

<div style="border:solid Green 2px; padding: 20px">

**Вывод по блоку:**
- В таблице **books** 1000 строк;
- В таблице **authors** 636 строки;
- В таблице **publishers** 340 строк;
- В таблице **ratings** 6456 строк;
- В таблице **reviews** 2793 строк;

## **Исследовательский анализ данных**

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [13]:
query = '''
            SELECT COUNT(book_id) 
            FROM books
            WHERE publication_date > '2000.01.01'
        '''
get_sql_data(query)

,count
0,819


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- После 1 января 2000 года вышло 819 книг.

### Посчитаем для каждой книги  количество обзоров и среднюю оценку;

In [14]:
query = '''
            SELECT b.title, 
                   COUNT(DISTINCT r.review_id) as count_review, 
                   ROUND(AVG(rat.rating), 1) as avg_rating
            FROM books b
            LEFT JOIN reviews r ON b.book_id = r.book_id
            LEFT JOIN ratings rat ON b.book_id = rat.book_id
            GROUP BY B.book_id
            ORDER BY count_review DESC
        '''
get_sql_data(query)

,title,count_review,avg_rating
0,Twilight (Twilight #1),7,3.7
1,Water for Elephants,6,4.0
2,The Glass Castle,6,4.2
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,The Curious Incident of the Dog in the Night-Time,6,4.1
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7
996,The Natural Way to Draw,0,3.0
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,Essential Tales and Poems,0,4.0


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- На первом месте расположились "Сумерки" с 7-ю обзорами и оценкой 3.7;
- На втором месте "Water for Elephants" с 6-ю обзорами и оценкой 4.0;
- На третьем месте "The Glass Castle" с 6-ю обзорами и оценкой 4.2.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [15]:
query = '''
            SELECT p.publisher, 
                   COUNT(b.book_id) AS count_book
            FROM publishers p
            LEFT JOIN books b on p.publisher_id = b.publisher_id
            WHERE num_pages > 50
            GROUP BY p.publisher
            ORDER BY count_book DESC
            LIMIT 1
        '''
get_sql_data(query)

,publisher,count_book
0,Penguin Books,42


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- Издательство **"Penguin Books"** выпустило 42 книги толще 50 страниц.

### Определим автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками)

In [16]:
query = '''

WITH 

a AS (
      SELECT book_id
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(rating_id) >= 50)

            SELECT au.author, 
                   ROUND(AVG(r.rating),7) AS avg_rating
            FROM books b
            JOIN a ON b.book_id = a.book_id
            JOIN authors au ON b.author_id = au.author_id
            JOIN ratings r ON a.book_id = r.book_id
            GROUP BY au.author_id, au.author
            ORDER BY AVG(r.rating) DESC
            LIMIT 2

        '''
get_sql_data(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- **J.K. Rowling/Mary GrandPré** - автор с самой высокой средней оценкой книг(4.29).

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
query = '''
        SELECT AVG (count_reviews) as avg_count_reviews
        FROM ( 
              SELECT username,
              COUNT(review_id) as count_reviews
              FROM reviews
              GROUP BY username
              HAVING username IN (
                                  SELECT username
                                  FROM ratings
                                  GROUP BY username
                                  HAVING COUNT (rating_id) > 48)) as reviews
            
        '''
get_sql_data(query).round(2)

,avg_count_reviews
0,24.0


<div style="border:solid Blue 2px; padding: 20px">

**Промежуточный вывод:**
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24

<div style="border:solid Orange 2px; padding: 20px">

**Общий вывод:**

**1. Загрузили библиотеки и установили параметры**;

**2. Ознакомились с данными:**
- В таблице **books** 1000 строк;
- В таблице **authors** 636 строки;
- В таблице **publishers** 340 строк;
- В таблице **ratings** 6456 строк;
- В таблице **reviews** 2793 строк;
 
**3. Провели исследовательский анализ данных:**
- После 1 января 2000 года вышло 819 книг;
- На первом месте расположились "Сумерки" с 7-ю обзорами и оценкой 3.7;
- На втором месте "Water for Elephants" с 6-ю обзорами и оценкой 4.0;
- На третьем месте "The Glass Castle" с 6-ю обзорами и оценкой 4.2;
- Издательство **"Penguin Books"** выпустило 42 книги толще 50 страниц;
- **J.K. Rowling/Mary GrandPré** - автор с самой высокой средней оценкой книг(4.29);
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24